<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🔨👨‍⚖️ תהליך הסבת שכבת גבולות פסקי דין לבנק"ל מודרני 🔨👨‍⚖️ 

<h1 style="text-align:center; font-size:150%">
תהליך זה הינו חלק 1 מתוך 2 אשר קורא את נתוני המקור, ממיר את המידע לפי אפיון נדרש ושומר את התוצאה בפורמט טבלאי 

In [ ]:
print('7. Judgments borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  7.a Reading inputs')

In [ ]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData"

today =  dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name

<h1 style="text-align:right;font-size:100%">
שכבת גבולות פס"ד קיימת

In [ ]:
LibDisplay_cols = ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'geometry']
LibDisplay_df = gpd.read_file(RawData_folder + '\CADSDE\GVUL_PSAK_DIN_NEW1.gdb', layer='GVUL_PSAK_DIN_NEW1', include_fields = LibDisplay_cols)

del LibDisplay_cols

<h1 style="text-align:right;font-size:100%">
טבלת פס"ד

In [ ]:
orcale_cols = ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'NAME', 'SOURCE', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'STATUS_DATE', 'START_DATE']
oracle_df = pd.read_csv(RawData_folder + "\Oracle\psak_din.csv", encoding = 'cp1255', usecols = orcale_cols)

del orcale_cols

<h1 style="text-align:right;font-size:100%">
שכבת גושים מודרנית

In [ ]:
Blocks_cols = ['BlockNumber', 'SubBlockNumber', 'BlockUniqueID']
Blocks = gpd.read_file(product_path, layer='Blocks', include_fields = Blocks_cols, ignore_geometry = True)
Blocks = Blocks.astype(int)


del [Blocks_cols, RawData_folder, today, date_name, product_path]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  7.b Cleaning data')

In [ ]:
print('  7.c Data conversion')

<h1 style="text-align:right;font-size:125%">
:עיבוד שכבת גבולות פס"ד
<h1 style="text-align:right;font-size:100%">
הוספת 4000 למספר פסק דין על מנת למנוע מספרים זהים לתהליכי קדסטר אחרים בשכבת גבולות קדסטר
<h1 style="text-align:right;font-size:100%">
הוספת שדה של שם התכנית לפי מספר ושנה בנתונים של אורקל ושל ליבדיספליי

In [ ]:
LibDisplay_df['PSAK_DIN_NUM'] = LibDisplay_df['PSAK_DIN_NUM'] + 4000
LibDisplay_df[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']] = LibDisplay_df[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']].astype(int)

LibDisplay_df['ProcessName'] = LibDisplay_df['PSAK_DIN_NUM'].astype(str) + '/' +	LibDisplay_df['PSAK_DIN_YEAR'].astype(str)
LibDisplay_df.drop(columns=['PSAK_DIN_NUM', 'PSAK_DIN_YEAR'], inplace=True)

<h1 style="text-align:right;font-size:125%">
:עיבוד טבלת פס"ד
<h1 style="text-align:right;font-size:100%">
הוספת 4000 למספר פסק דין על מנת למנוע מספרים זהים לתהליכי קדסטר אחרים בשכבת גבולות קדסטר
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם תהליך
<h1 style="text-align:right;font-size:100%">
הגדרת שדותשל תאריך כשדות מסוג תאריך

In [ ]:
oracle_df['PSAK_DIN_NUM'] = oracle_df['PSAK_DIN_NUM'] + 4000
oracle_df['ProcessName'] = oracle_df['PSAK_DIN_NUM'].astype('str') + '/' +	oracle_df['PSAK_DIN_YEAR'].astype('str')

In [ ]:
oracle_df['STATUS_DATE'] = oracle_df['STATUS_DATE'].astype('datetime64[ns]')
oracle_df['START_DATE'] = oracle_df['START_DATE'].astype('datetime64[ns]')

<h1 style="text-align:right;font-size:125%">
:עיבוד שכבת גושים מודרנית
<h1 style="text-align:right;font-size:100%">
יצירת שדה מקשר לטובת איחוד עם שכבת גושים והשגת מזהה גוש בהמשך   

In [ ]:
Blocks['BlockName'] = Blocks['BlockNumber'].astype(str) + '_' + Blocks['SubBlockNumber'].astype(str)

oracle_df['GUSH_SUFFIX'].fillna(0, inplace=True)
oracle_df[['GUSH_NUM', 'GUSH_SUFFIX']] = oracle_df[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
oracle_df['BlockName'] = oracle_df['GUSH_NUM'].astype(str) + '_' + oracle_df['GUSH_SUFFIX'].astype(str)

<h1 style="text-align:right;font-size:125%">
מיזוג טבלאות 

In [ ]:
merge = LibDisplay_df.merge(oracle_df, on = 'PSAK_DIN_ID', how = 'outer')
merge.drop(columns = ['ProcessName_x'], inplace = True)
merge.rename(columns = {'ProcessName_y' : 'ProcessName'}, inplace=True)

In [ ]:
merge = merge.merge(Blocks, on = 'BlockName', how = 'left')
merge = merge.drop_duplicates(keep='first')

del [LibDisplay_df, oracle_df, Blocks]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  7.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                       #("BlockUniqueID", int), יאוחד בהמשך התהליך
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),#לא קיים שדה לייבוא
                        ("Status", int),
                        ("PlanName", str),
                        ("SurveyorLicenseID", int), #לא קיים שדה לייבוא
                        ("DataSource", int),
                        ("geometry", int)
                        ])


Judgments = gpd.GeoDataFrame(np.empty(0, dtype = new_columns),
                             crs = 2039,
                             geometry = 'geometry')

del new_columns

<h1 style="text-align:right;font-size:100%">
    איכלוס שדות

In [ ]:
Judgments['BlockUniqueID'] = merge['BlockUniqueID']


Judgments['ProcessType'] = 3


Judgments['MapYear'] = merge['START_DATE'].dt.year
Judgments['MapYear'] = Judgments['MapYear'].astype('Int64')


Judgments['ProcessName'] = merge['ProcessName'].astype(str)


statusDict = {1 : 31,
              2 : 32,
              3 : 33,
              4 : 36,
              None : 36}
Judgments['Status'] = merge['STATUS'].map(statusDict)
Judgments['Status'] = Judgments['Status'].fillna(36)
Judgments['Status'] = Judgments['Status'].astype(int)


Judgments['PlanName'] = None


Judgments['GeodeticNetwork'] = None


Judgments['SurveyorLicenseID'] = None


Judgments['DataSource'] = 3


Judgments['geometry'] = merge['geometry']


Judgments['CPBUniqueID'] = merge['PSAK_DIN_ID'] + 4000000
Judgments.dropna(subset=['CPBUniqueID'], inplace=True)
Judgments['CPBUniqueID'] = Judgments['CPBUniqueID'].astype(int)

In [ ]:
Judgments.sort_values('CPBUniqueID', inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
Judgments_geom    = Judgments[~Judgments['geometry'].isna()]
Judgments_no_geom = Judgments[Judgments['geometry'].isna()]

In [ ]:
print('  7.e Exporting results to product folder')

In [ ]:
#geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\Judgments_geom.csv'
no_geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\Judgments_no_geom.csv'
json_dir =  r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\Judgments_geom.json'


#Judgments_geom.to_csv(geom_csv_dir, encoding = 'cp1255', index=False)
Judgments_no_geom.to_csv(no_geom_csv_dir, encoding = 'cp1255', index=False)

Judgments_geom.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"\\mapi_shares\MNCDB\צוות מידע\scripts\arcpyPro\Judgments-Save as Feature Class.bat"]);